In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import copy
import glob
import random
from random import random as rd
import gc

In [ ]:
df_train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
unique_pressures = df_train["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)
def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        return sorted_pressures[-1]
    elif insert_idx == 0:
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val
def set_seed(seed = 2021):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state
def wc(input_list):
    l = []
    for i in range(len(input_list)):
        public_lb_score = int(input_list[i].split("/")[-1].split(".")[1].split(" ")[0]) 
        l.append(public_lb_score)
        input_list[i] = (pd.read_csv(input_list[i]).pressure).ravel()
    output = 0
    l_sum = sum(l)
    if len(input_list) == 1:
        output = input_list[0]
    else:
        weight1 = (l[1] / l_sum) + 0.1
        weight2= 1 - weight1
        output += input_list[0] * weight1 + input_list[1] * weight2
    return output
def g(dp):
    l = []
    for i in glob.iglob(f'{dp}/*'):
        l.append(i)
    file_count = len(l)
    loop_time = 154
    splits = file_count // 2
    l.sort()
    flist = []
    for i in range(splits):
        if i == splits - 1:
            flist.append(l[i * round(len(l) / splits): ])
        else:
            flist.append(l[i * round(len(l) / splits): (i + 1) * round(len(l) / splits)])
    for i in range(len(flist)):   
        flist[i] = wc(flist[i])
    pred_list = []
    for i in range(loop_time):      
        weight = []        
        set_seed(i)
        for i in range(len(flist)):
            weight.append(rd())  
        weight_sum = sum(weight)
        for i in range(len(weight)):
            weight[i] /= weight_sum
        weight.sort(reverse = True)
        temp = 0
        for i in range(len(flist)):
            temp += flist[i] * weight[i]
        pred_list.append(temp)
        del temp
        gc.collect()
    output = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
    median_pred = np.median(np.vstack(pred_list), axis = 0)
    mean_pred = sum(pred_list) / len(pred_list)
    output.pressure = 0.8 * median_pred + 0.2 * mean_pred
    output["pressure"] = output["pressure"].apply(find_nearest)
    output.to_csv(f'rwb {loop_time} loops.csv',index=False)
def blend(a, b):
    a = pd.read_csv(a)
    b = pd.read_csv(b)
    a.pressure = a.pressure * 0.7 + b.pressure * 0.3
    a["pressure"] = a["pressure"].apply(find_nearest)
    a.to_csv('blend.csv', index = False)
    return a
def avg(dp):
    input_list = []
    for i in glob.iglob(f'{dp}/*'):
        input_list.append(i)
    for i in range(len(input_list)):
        input_list[i] = (pd.read_csv(input_list[i]).pressure).ravel()
    output = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
    output.pressure = np.median(np.vstack(input_list), axis = 0)
    output["pressure"] = output["pressure"].apply(find_nearest)
    output.to_csv('avg.csv', index = False)
    return output

In [ ]:
# avg('../input/gb-data-blending-recover')

In [ ]:
g('../input/gb-data-blending-recover')

In [ ]:
# a = '../input/gb-data-blending-recover/0.1341 blend.csv'
# b = '../input/gb-data-blending-recover/0.1349.csv'
# blend(a, b)